# Processing Images and Videos
## Computer Vision and Image Processing - Lab Session 3
### Prof: Luigi Di Stefano, luigi.distefano@unibo.it
### Tutor: Pierluigi Zama Ramirez, pierluigi.zama@unibo.it - Alex Costanzino, alex.costanzino2@unibo.it

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Processing Images

## Histogram

Point Operators are image processing operators aimed at enhancing the quality of the input image(e.g. the contrast).
Most of such operators rely on the computation of the gray-level histogram (intensity histogram) of the input image.
Loaded a **grayscale** image:

In [ ]:
img = cv2.imread("LabSession3Images/image.jpg", cv2.IMREAD_GRAYSCALE)
plt.imshow(img, cmap='gray', vmin=0, vmax=255)
plt.show()

We can compute its gray-level histogram using **np.histogram(flat_image, bins=256, range=[0,256])**

Where:
* **flat_image**: image flattened to be a 1D array
* **bins**: bounds of the bins. If bins is an integer, the bounds are all numbers from minimum to maximum (exluded). Otherwise, they can be also a custom sequence.
* **range**: minimum and maximum values of our histograms.

Bins will have 257 elements, because Numpy calculates bins as 0-0.99, 1-1.99, 2-2.99 etc. So final range would be 255-255.99. The last value 256 is the last edge of the last bin but it is excluded.

In [ ]:
hist, bins = np.histogram(img.flatten(), 256, [0,256])
print("Histogram shape: ", hist.shape)
print("Bins shape: ", bins.shape)

Where __hist[i]__ are the number of pixels with the gray-scale value $i$. Bins are the indexes of each bin (i.e. numbers from 0 to 255).

Let us see how many pixels have the value 20 in our image:

In [ ]:
grayscale_value = 20
print(hist[grayscale_value])

Let us plot the histogram:

In [ ]:
plt.stem(hist, use_line_collection=True)
plt.show()

## Example of Point Operator: Linear Contrast Stretching

An image featuring a small gray-level range has likely poor contrast. The contrast can be enhanced by linearly stretching the intensities to span a larger interval (typically, the full available range).

Linear stretching at the full available range:

<img src="LabSession3Images/linear_stretching_plot.png" width="320">
<img src="LabSession3Images/linear_stretching.png" width="480">

Should most pixel lie in a small interval while there exist a few dark and bright outliers, the linear function would approximate the identity and thus be ineffective. In such a case, $r_{min}$ and $r_{max}$ can be taken equal to some percentiles of the distribution (e.g. 5% and 95%), with smaller and higher input intensities mapped to 0 and 255 respectively.

## Local Operators: Convolutions and Correlations 

Local Operators compute the new intensity of a pixel, p, based on the intensities of those belonging to a neighbourhood of p.

<img src="LabSession3Images/local_op.png" width="320">

An important sub-class is given by the so called **Linear Shift-Invariant** (LSI) operators.
LSI operators are defined **convolving** a kernel with an image.

However, we know that **convolutions and correlations** are the same if we have a **symmetric** kernel respect to the **origin**:

<img src="LabSession3Images/conv_corr.png" width="128">

Practically, local operator kernels are usually symmetric about the origin so usually we compute **correlations** instead of convolutions because they are **easier to implement**. In case we have not symmetric filter respect to the origin we can still use correlations but we need to first **flip the kernel**.

Given an image and a kernel (aka filter) in OpenCV we can perform a correlation with the following command:

**cv2.filter2D(image, -1, kernel, anchor)**

where **anchor** of the kernel indicates the relative position of a filtered point within the kernel; the anchor should lie within the kernel; default value (-1,-1) means that the anchor is at the kernel center.

If we need to compute a convolution we need to flip the kernel using:

**cv2.flip(kernel)**

In case of flip we need to set the new anchor to (kernel.cols - anchor.x - 1, kernel.rows - anchor.y - 1).

## Local Operators: Some Peculiar Filters

### Gaussian filter
Gaussian filter is a special case of LSI operator where we sample from a 2D gaussian the values of our kernel.  

<img src="LabSession3Images/gaussian.png" width="320">

Given standard deviation $\sigma$ and kernel size we can obtain a 1D gaussian kernel in OpenCV with:

**gk = cv2.getGaussianKernel(kernel_size,sigma)**

which 1D having shape $(KernelSize , 1)$

If we want a 2D gaussian filter with both $\sigma_x$ and $\sigma_y$ having the same value we can multiply the two 1D kernels with:

**gk_2D = gk.dot(gk.transpose())**

Otherwise, we can obtain the same result applying first the 1D gaussian kernel **gk** and then, applying its tranposed one **gk$^T$**. Applying twice a 1D kernel instead of a 2D filter can be faster, expecially in case of big kernels.

We can also use an high level API of OpenCV which given the kernel size and sigmas compute the gaussian filtering:

**gauss_out = cv2.GaussianBlur(image, (k_size,k_size) , sigma)**

### Bilateral filter

<img src="LabSession3Images/bilateral.png" width="512">

Bilateral filter is highly effective at **noise removal** while **preserving edges**. 

The operation is **slower** compared to other filters. 

**Gaussian filters** is a function of **space** alone and does not consider whether pixels have almost the same intensity value or not resulting in blurred edges. 

On the other hand, the **bilateral filter** also uses a **Gaussian filter** in the **space** domain, but it also uses one more (multiplicative) **Gaussian filter** component which is a function of **pixel intensity differences**. 

The Gaussian function of space makes sure that only pixels are ‘spatial neighbors’ are considered for filtering, while the Gaussian component applied in the intensity domain (a Gaussian function of intensity differences) ensures that only those pixels with intensities similar to that of the central pixel (‘intensity neighbors’) are included to compute the blurred intensity value. As a result, this method preserves edges, since for pixels lying near edges, neighboring pixels placed on the other side of the edge, and therefore exhibiting large intensity variations when compared to the central pixel, will not be included for blurring.

We can apply a bilateral filter in OpenCV with:

**cv2.bilateralFilter(image, filter_size, sigmaColor, sigmaSpace)**.

# Processing Videos

A video is a temporal sequence of images, namely **Frames**.

<img src="LabSession3Images/Eadward-Muybridge-Horse-in-Motion.jpg" width="512">

The number of frames in 1 second of video is called **frame rate**

In a video we can elaborate each frame separately with processing algorithms or we can elaborate only keyframe of the original video and the missing ones are reconstructed by interpolation.

## Load a Video from File

To load a video in OpenCV you need to create a VideoCapture object. Its argument can be either the device index or the name of a video file.
A device index is just the number to specify which camera. Normally one camera will be connected (as in my case). So I simply pass 0 (or -1).
You can select the second camera by passing 1 and so on. After that, you can capture frame-by-frame. But at the end, don't forget to release the capture.

Let us now try to load a video from a file:

In [ ]:
cap = cv2.VideoCapture('LabSession3Images/video.avi')

Sometimes, cap may not have initialized the capture. In that case, this code shows an error. You can check whether it is initialized or not by the method cap.isOpened(). If it is True, OK. Otherwise open it using cap.open().

In case the capture is open we can get a frame of the capture in the following way:

In [ ]:
if cap.isOpened():
    ret, frame = cap.read()

print(ret, frame.shape)

Where _ret_ is a boolean variable which is True if we read the frame correctly and _frame_ is an image. Let us visualize the captured frame:

In [ ]:
if ret and not frame is None:
    # Disabling matplotlib axis for better visualization
    plt.axis('off')
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.show()

You can also access some of the features of this video using cap.get(propId) method where propId is a number from 0 to 18. Each number denotes a property of the video (if it is applicable to that video). Some of these values can be modified using cap.set(propId, value).

For example to get the width and height of the frame:

In [ ]:
print("Width: " , cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print("Height: ", cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

My video is of resolution 320x180.

To release the capture stream:

In [ ]:
cap.release()

If we want to play our video in **jupyter notebook** we can do the following (a bit slow because of jupyter not ideal for playing videos):

In [ ]:
# Import additional library to properply play videos on jupyter notebook
from IPython.display import clear_output

# Put the code in try-except statements catch the keyboard exception and release the camera device and 
# continue with the rest of code.
def play_video(video_path):
    cap = cv2.VideoCapture(video_path)
    # Try-except statement to manage exceptions
    try:
        while(True):
            # Capture frame
            ret, frame = cap.read()
            if not ret or frame is None:
                # Release the Video if ret is false
                cap.release()
                print("Released Video Resource")
                # Break exit the for loops
                break
            
            # Display the frame
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            plt.axis('off')
            plt.title("Input Stream")
            plt.imshow(frame)
            plt.show()
            
            # Clear cell output when new frame is available
            clear_output(wait=True)
    except KeyboardInterrupt:
        # If we press stop (jupyter GUI) release the video
        cap.release()
        print("Released Video Resource")

play_video('LabSession3Images/video.avi')

__N.B__: It can happen that we want to play a video from a non-jupyter enviroment. In that case we can modify the above code as follows:

```python
def play_video(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret or frame is None:
            # Release the Video if ret is false
            cap.release()
            print("Released Video Resource")
            break

        # !!! DISPLAYING CHANGE RESPECT TO JUPYTER VERSION !!!
        # Displaying with OpenCV (Not working in Jupyter)
        cv2.imshow('frame', frame)
        # Stop playing when entered 'q' from keyboard
        if cv2.waitKey(1) == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
```

## Load a Video from a Camera (Only if Camera Available)
If we want to load a video from a camera we can open our video capture giving a device id as input.

In [ ]:
# Working only if a camera is available.
cap = cv2.VideoCapture(0)

print("Width: " , cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print("Height: ", cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

Resolution is 640x480 by default. We can change it using cap.set (not all resolutions are possible depending on the camera driver):

In [ ]:
ret = cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)

Now frames have the desired resolution:

In [ ]:
if cap.isOpened():
    ret, frame = cap.read()

print(ret, frame.shape)

cap.release()

Let us try to reproduce our webcam stream:

In [ ]:
play_video(0)

## Saving a Video

To save a video on a file we need to create a VideoWriter object specifying:
* **Filename** (eg: output.avi)
* **FourCC code**: 4-byte code used to specify the video codec. The list of available codes can be found in fourcc.org. It is platform dependent. In Windows DIVX is the preferred choice while on linux we have several standards such as DIVX, XVID, X264 etc..
* **FPS**: number of frames per second
* **Frame size**
* Flag **isColor**: If it is True, the encoder expect color frame, otherwise it works with grayscale frame.

Let us try to load and save a video with each frame flipped along vertical axis. Load the original video:

In [ ]:
# Loading Video
cap = cv2.VideoCapture("LabSession3Images/video.avi")

# Getting original video params
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

Initialize the VideoWriter based on the parameters of the original video:

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
# N.B. we need to specify the correct width and height of the frames otherwise we will not be able to reproduce the video
out = cv2.VideoWriter('output.avi', fourcc, fps, (w,  h))

Flip and save frame by frame the video:

In [ ]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret or frame is None:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    frame = np.flip(frame, axis=0)
    # write the flipped frame
    out.write(frame)

Release the resources:

In [ ]:
# Release everything if job is finished
cap.release()
out.release()

Let us visualize the flipped video:

In [ ]:
play_video('output.avi')